# Implement our own system assembling

In this tutorial we

* write integrators for $\int_T f v dx$ and $\int_T \nabla u \nabla v dx$: 
* put together element matrices to the global system matrix: 


The implementation is available on github: 
[https://github.com/TUWien-ASC/NGS-myassembling](https://github.com/TUWien-ASC/NGS-myassembling)



In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
import myassembling
dir (myassembling)

Loading myassembling library


['MyAssembleMatrix',
 'MyLaplace',
 'MySource',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=1, dirichlet=".*")
u, v = fes.TnT()

### use our own integrators for element matrix calculation:

In [3]:
a = BilinearForm(fes)
a += myassembling.MyLaplace(1)
a.Assemble()

f = LinearForm(fes)
f += myassembling.MySource(x)
f.Assemble();

In [4]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

### we can inspect the element matrix:

In [5]:
mylap = myassembling.MyLaplace(1)
ei = ElementId(VOL,17)
mylap.CalcElementMatrix(fes.GetFE(ei), mesh.GetTrafo(ei))

 0.665986 -0.426433 -0.239553
 -0.426433 0.648429 -0.221997
 -0.239553 -0.221997 0.46155

### use our own matrix assembling function:

In [ ]:
for l in range(4): mesh.Refine()
fes.Update()
gfu.Update()
f.Assemble();

In [ ]:
mymatrix = myassembling.MyAssembleMatrix(fes, myassembling.MyLaplace(CF(1)))

if fes.ndof < 100000:
    gfu.vec.data = mymatrix.Inverse(fes.FreeDofs()) * f.vec
    Draw (gfu);

### Exercise:


   * Implement `MyAssembleVector` for building the right hand side vector for  $\int_\Omega f v \, dx$
   * Implement `MyNeumannIntegrator` for evaluating $\int_{\partial \Omega} g v \, ds$